<a href="https://colab.research.google.com/github/cs-amy/project-codebase/blob/main/notebooks/Character_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model for Character De-Obfuscation**
Stage 1 of MSc Project — Ashraf Muhammed Yusuf

# **1. Colab Environment Setup**

In [ ]:
# Install dependencies
!pip install -q tensorflow matplotlib

# Import dependencies
import os
import glob
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
from tensorflow.keras import layers, models, callbacks, mixed_precision
from sklearn.metrics import classification_report, confusion_matrix

# Mount Drive so you can read datasets and write checkpoints
# Link to dataset:
# https://drive.google.com/drive/folders/1sfNG1PkmTPBe1wOSQXZmfdkvR97Hn9lk?usp=sharing
drive.mount('/content/drive')

# (Optional—but useful) turn on XLA JIT for extra speed
tf.config.optimizer.set_jit(True)

# **2. Data Loading & Splitting**
We'll use Keras's image_dataset_from_directory to build train/validation and test sets.

In [ ]:
BATCH = 64
IMG_SIZE = (64, 64)
BASE_DIR="/content/drive/MyDrive/MScProject"
CKPT_DIR=f"{BASE_DIR}/char_ckpt_best.keras"
train_dir = f"{BASE_DIR}/data/characters/train"
test_dir = f"{BASE_DIR}/data/characters/test"

# Train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SIZE,
  color_mode="grayscale",
  validation_split=0.20,
  subset="training",
  seed=42
)

# Val dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SIZE,
  color_mode="grayscale",
  validation_split=0.20,
  subset="validation",
  seed=42
)

# Test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SIZE,
  color_mode="grayscale",
  shuffle=False
)

In [ ]:
# Utility to display examples from each set
def show_examples(ds, ds_name, num=5):
  # Take one batch
  for images, labels in ds.take(1):
    images = images.numpy()
    labels = labels.numpy()
    class_names = ds.class_names
    break

  plt.figure(figsize=(6,6))
  for i in range(num):
    ax = plt.subplot(3, 3, i+1)
    img = images[i].squeeze()  # shape: (H,W) since grayscale
    lbl = class_names[labels[i].argmax()]
    plt.imshow(img, cmap='gray')
    plt.title(f"{ds_name}: {lbl}")
    plt.axis('off')
  plt.tight_layout()
  plt.show()

# Display 5 examples from each split
show_examples(train_ds, "Train")
show_examples(val_ds, "Val")
show_examples(test_ds, "Test")

# **3. Preprocessing & Augmentation**

In [ ]:
# Save dataset class names before piping the dataset through 'map'
train_ds_class_names = test_ds.class_names
val_ds_class_names = test_ds.class_names
test_ds_class_names = test_ds.class_names

# Normalize and augment datasets (only the train dataset is augmented)
normalization = layers.Rescaling(1./255)
aug = tf.keras.Sequential([
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
  layers.RandomTranslation(0.1, 0.1)
])

train_ds = train_ds.map(lambda x,y: (aug(normalization(x)), y))
val_ds   = val_ds.map(lambda x,y: (normalization(x), y))
test_ds  = test_ds.map(lambda x,y: (normalization(x), y))

In [ ]:
!pip install --upgrade --force-reinstall pydot
!sudo apt-get update -qq
!sudo apt-get install -y graphviz

# **4. Model Architecture**
We will define a simple yet robust CNN (grayscale)

In [ ]:
# Define model

# 1. Input Layer
inputs = layers.Input(shape=(*IMG_SIZE, 1))

# 2. (Conv + ReLU) + Pooling 1
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D()(x)

# 3. (Conv + ReLU) + Pooling 2
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)

# 4. (Conv + ReLU)
x = layers.Conv2D(128, 3, activation='relu')(x)

# 5. Flatten to Vector
x = layers.Flatten()(x)

# 6. (FC + ReLU) Layer
x = layers.Dense(128, activation='relu')(x)

# 7. Dropout Regularisation
x = layers.Dropout(0.5)(x)

# 8. Output Layer
outputs = layers.Dense(26, activation='softmax')(x)

# Construct model
model = models.Model(inputs, outputs)

from tensorflow.keras.utils import plot_model
from IPython.display import Image, display

# Show model summary
model.summary()




# **5. Compilation & Callbacks**

In [ ]:
# Compile model
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-3),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Callbacks
# 1. Checkpoint - saves the best model
ckpt = callbacks.ModelCheckpoint(
  filepath=CKPT_DIR,
  save_best_only=True,
  monitor="val_loss" # keep only the best model
)
# 2. Early stopping
es = callbacks.EarlyStopping(
  monitor="val_loss",
  patience=6, # stop ~6 epochs after val_loss stalls
  restore_best_weights=True
)

# 3. LR scheduler
lr_s = callbacks.ReduceLROnPlateau(
  monitor="val_loss",
  factor=0.5,
  patience=3, # halve LR if val_loss hasn’t improved for 3 epochs
  min_lr=1e-6
)

callbacks=[ckpt, es, lr_s]

# **6. Training**

In [ ]:
# Train model
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=50,
  callbacks=callbacks
)

# **7. Evaluation & Visualisation**

In [ ]:
# Load best checkpoint's weights
model.load_weights(CKPT_DIR)

# Test model accuracy on test dataset
model.evaluate(test_ds)

# Training curves
epochs = range(1, len(history.history['loss']) + 1)
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['accuracy'],    label='train_acc')
plt.plot(epochs, history.history['val_accuracy'],label='val_acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['loss'],    label='train_loss')
plt.plot(epochs, history.history['val_loss'],label='val_loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()

# **8. Confusion Matrix & Classification Report**

In [ ]:
# Gather all ground-truths and predictions
y_true = []
y_pred = []
for batch_x, batch_y in test_ds:
  preds = model.predict(batch_x)
  y_pred.extend(np.argmax(preds, axis=1))
  y_true.extend(np.argmax(batch_y.numpy(), axis=1))

class_names = test_ds_class_names
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(train_ds_class_names))
plt.xticks(tick_marks, train_ds_class_names, rotation=90)
plt.yticks(tick_marks, train_ds_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

# **9. Display Misclassified Examples**

In [ ]:
# First flatten all test images to a single array
images_all = np.concatenate([x.numpy() for x, y in test_ds], axis=0)
mis_idx = [i for i, (t, p) in enumerate(zip(y_true, y_pred)) if t!=p]

plt.figure(figsize=(9, 9))
for i, idx in enumerate(mis_idx[:9]):
  plt.subplot(3, 3, i+1)
  img = images_all[idx].squeeze()
  plt.imshow(img, cmap='gray')
  plt.title(f"T:{class_names[y_true[idx]]} P:{class_names[y_pred[idx]]}")
  plt.axis('off')
plt.tight_layout()
plt.show()